<a href="https://colab.research.google.com/github/aipatchouli/JAX_Demo/blob/main/CNN/CNN_JAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/googledrive')

Mounted at /content/googledrive


In [3]:
!pip install -Uq tensorflow-datasets jax optax flax dm-haiku rlax tensorflow
!pip show jax | grep "Version"
!cat /proc/cpuinfo | grep "model name"
!nvidia-smi

Version: 0.4.10
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
Sat May 27 13:13:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--------

In [4]:
import jax
jax.devices()

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]

In [5]:
from jax import jit, grad, vmap
import jax.numpy as jnp
from jax import random
import numpy as np

key = random.PRNGKey(10007)

In [ ]:
# 数据集
import tensorflow_datasets as tfds
import tensorflow as tf

def get_datasets_MNIST(num_epochs, batch_size):
    train_ds = tfds.load('mnist', split='train')
    test_ds = tfds.load('mnist', split='test')

    train_ds = train_ds.map(lambda sample: {
        'image': tf.cast(sample['image'], tf.float32) / 255.,   # normalize
        'label': sample['label']
    })
    test_ds = test_ds.map(lambda sample: {
        'image': tf.cast(sample['image'], tf.float32) / 255.,
        'label': sample['label']
    })

    train_ds = train_ds.repeat(num_epochs).shuffle(1024) 
    train_ds = train_ds.batch(batch_size, drop_remainder=True).prefetch(1)
    test_ds = test_ds.shuffle(1024)
    test_ds = test_ds.batch(batch_size, drop_remainder=True).prefetch(1)

    return train_ds, test_ds
